Solve system:
$$
\mathbf{\|B_k X - T_k\|_F → min_X}
$$
where
$$
\mathbf{B_k = ⊙_{i != k} factors[i]}
$$
$$
\mathbf{X = factors[k]}
$$
$$
\mathbf{T_k} \textbf{- unfolder k of tensor}
$$
So we have to solve:
$$
\mathbf{B_k^* B_k X = B_k^*T_k}
$$
$$
\mathbf{B_k^* B_k = \circ_{i!= k} U_i^T U_i}
$$
Apply Cholesky
$$
\mathbf{L_k^* L_k X = B_k^*T_k}
$$
$$
\mathbf{L_k^* X = L_k^{-1} B_k^*T_k}
$$
And then find X

In my code we try to calculate fast $U_n^T U_n$ , $\circ_{i= n-1, n} U_i^T U_i \ldots$ and next adamar products older than k (BB)\
Values, which smaller than k we calculate in ad_product\
Analogious we calculate B in (B) and hr_product

In [1]:
import numpy as np
from scipy import linalg as lg
import time

In [221]:
def ALS(tensor, rank, seed=42, tol = 10**(-8), n_iteration = 1000):

    dimensions = tensor.shape
    len_dim = len(dimensions)

    rng = np.random.RandomState(seed)
    factors = np.array([rng.rand(dimensions[i], rank) for i in range(len_dim)], dtype=object)

    eps = 1.0
    it = 0

    while eps > tol and it < n_iteration:

        eps = 0.0

        BB = [factors[len_dim - 1].T @ factors[len_dim - 1]]
        B = [factors[len_dim - 1]]
        for i in range(1, len_dim - 1):
            BB.append((factors[len_dim - 1 - i].T @ factors[len_dim - 1 - i]) * BB[i - 1]) #len_dim - 1
            B.append(lg.khatri_rao(factors[len_dim - 1 - i], B[i - 1]))

        ad_prod = np.ones((rank, rank))
        hr_prod = np.ones((1, rank))

        permutat = np.arange(1, len_dim)
        permutat = np.append(permutat, 0)

        for i in range(len_dim):

            if i != 0 and i != len_dim - 1:
                BB_i = ad_prod * BB[len_dim - 2 - i]
                B_i = lg.khatri_rao(hr_prod, B[len_dim - 2 - i]).T
            elif i == 0:
                BB_i = BB[len_dim - 2]
                B_i = B[len_dim - 2].T
            else:
                BB_i = ad_prod
                B_i = hr_prod.T

            L = lg.cholesky(BB_i, lower=True)

            B_i = B_i @ np.transpose(tensor, permutat).reshape(-1, dimensions[i])

            y = lg.solve_triangular(L, B_i, lower=True)

            new_factor = (lg.solve_triangular(L.T, y, lower=False)).T

            eps = max(eps, np.linalg.norm(new_factor - factors[i]) / np.linalg.norm(factors[i]))

            hr_prod = lg.khatri_rao(hr_prod, new_factor)
            ad_prod *= new_factor.T @ new_factor

            factors[i] = new_factor

            permutat[i], permutat[-1] = permutat[-1], permutat[i]

        it += 1

    return factors, it

In [ ]:
def fast_ALS_one(tensor, rank, seed=42, tol = 10**(-8), n_iteration = 100):

    dimensions = tensor.shape
    len_dim = len(dimensions)

    rng = np.random.RandomState(seed)
    factors = np.array([rng.rand(dimensions[i], rank) for i in range(len_dim)], dtype=object)

    eps = 1.0
    it = 0

    while eps > tol and it < n_iteration:

        eps = 0.0

        permutat = np.arange(len_dim)

        BB = [factors[len_dim - 1].T @ factors[len_dim - 1]]
        BT = [tensor.reshape(-1, dimensions[len_dim - 1]) @ factors[len_dim - 1]]

        for i in range(1, len_dim - 1):
            BB.append((factors[len_dim - 1 - i].T @ factors[len_dim - 1 - i]) * BB[i - 1])

            prom = BT[i - 1].reshape(dimensions[:(len_dim - i)]).reshape(-1, dimensions[len_dim - 1 - i])
            BT.append(prom @ factors[len_dim - 1 - i])

        ad_prod = np.ones((rank, rank))

        for i in range(len_dim):

            if i != 0 and i != len_dim - 1:
                BB_i = ad_prod * BB[len_dim - 2 - i]
                small_permute = np.array([i])
                small_permute = np.append(small_permute, np.arange(i))
                small_dim = np.array(dimensions[i])
                small_dim = np.append(small_dim, dimensions[:i])
                B_i = np.transpose(BT[len_dim - 2 - i].reshape(dimensions[:(i + 1)]), small_permute)
                for j in range(i - 1, -1, -1):
                    B_i = B_i.reshape(-1, dimensions[j]) @ factors[j]
                    B_i = B_i.reshape(small_dim[:(j + 1)])
                    small_permute[0] -= 1
                B_i = B_i.reshape(small_dim[0], -1)


            elif i == 0:
                BB_i = BB[len_dim - 2]
                B_i = BT[len_dim - 2]
            else:
                BB_i = ad_prod

                small_permute = np.array([len_dim - 1])
                small_permute = np.append(small_permute, np.arange(len_dim - 1))
                small_dim = np.array(dimensions[len_dim - 1])
                small_dim = np.append(small_dim, dimensions[:len_dim - 1])

                B_i = np.transpose(tensor, small_permute)

                for j in range(len_dim - 1, 0, -1):
                    B_i = B_i.reshape(-1, small_dim[j]) @ factors[j - 1]
                    B_i = B_i.reshape(small_dim[:j])
                    small_permute[0] -= 1

                B_i = B_i.reshape(small_dim[0], -1)

            L = lg.cholesky(BB_i, lower=True)

            y = lg.solve_triangular(L, B_i.T, lower=True)

            new_factor = (lg.solve_triangular(L.T, y, lower=False)).T

            eps = max(eps, np.linalg.norm(new_factor - factors[i]) / np.linalg.norm(factors[i]))

            ad_prod *= new_factor.T @ new_factor

            factors[i] = new_factor

            permutat[i], permutat[-1] = permutat[-1], permutat[i]



        it += 1


    return factors, it

In [222]:
def fast_ALS(tensor, rank, seed=42, tol = 10**(-8), n_iteration = 1000):

    dimensions = tensor.shape
    len_dim = len(dimensions)

    rng = np.random.RandomState(seed)
    factors = np.array([rng.rand(dimensions[i], rank) for i in range(len_dim)], dtype=object)

    eps = 1.0
    it = 0

    while eps > tol and it < n_iteration:

        eps = 0.0

        permutat = np.arange(len_dim)

        BB = [factors[len_dim - 1].T @ factors[len_dim - 1]]
        prom = np.array(tensor.reshape(-1, dimensions[len_dim - 1]) @ factors[len_dim - 1][:, 0])
        for r in range(1, rank):
            prom = np.vstack((prom, np.array(tensor.reshape(-1, dimensions[len_dim - 1]) @ factors[len_dim - 1][:, r])))
        BT = [prom.T]

        for i in range(1, len_dim - 1):
            BB.append((factors[len_dim - 1 - i].T @ factors[len_dim - 1 - i]) * BB[i - 1])
            prom = np.array(BT[i - 1][:, 0].reshape(dimensions[:(len_dim - i)]).reshape(-1, dimensions[len_dim - 1 - i]) @ factors[len_dim - 1 - i][:, 0])

            for r in range(1, rank):
                prom = np.vstack((prom, np.array(BT[i - 1][:, r].reshape(dimensions[:(len_dim - i)]).reshape(-1, dimensions[len_dim - 1 - i]) @ factors[len_dim - 1 - i][:, r])))

            BT.append(prom.T)

        ad_prod = np.ones((rank, rank))

        for i in range(len_dim):

            if i != 0 and i != len_dim - 1:
                BB_i = ad_prod * BB[len_dim - 2 - i]

                small_permute = np.array([i])
                small_permute = np.append(small_permute, np.arange(i))
                small_dim = np.array(dimensions[i])
                small_dim = np.append(small_dim, dimensions[:i])

                B_i = None
                for r in range(rank):
                    prom = np.transpose(BT[len_dim - 2 - i][:, r].reshape(dimensions[:(i + 1)]), small_permute)
                    for j in range(i - 1, -1, -1):
                        prom = prom.reshape(-1, dimensions[j]) @ factors[j][:, r]
                        prom = prom.reshape(small_dim[:(j + 1)])
                        small_permute[0] -= 1
                    if B_i is None:
                        B_i = prom
                    else:
                        B_i = np.vstack((B_i, prom))
                    small_permute[0] = i

                B_i = B_i.T

            elif i == 0:
                BB_i = BB[len_dim - 2]
                B_i = BT[len_dim - 2]
            else:
                BB_i = ad_prod

                small_permute = np.array([len_dim - 1])
                small_permute = np.append(small_permute, np.arange(len_dim - 1))
                small_dim = np.array(dimensions[len_dim - 1])
                small_dim = np.append(small_dim, dimensions[:len_dim - 1])

                B_i = None
                for r in range(rank):
                    small_permute[0] = len_dim - 1
                    prom = np.transpose(tensor, small_permute)
                    for j in range(len_dim - 1, 0, -1):
                        prom = prom.reshape(-1, small_dim[j]) @ factors[j - 1][:, r]
                        prom = prom.reshape(small_dim[:j])
                        small_permute[0] -= 1
                    if B_i is None:
                        B_i = prom
                    else:
                        B_i = np.vstack((B_i, prom))
                B_i = B_i.T

            L = lg.cholesky(BB_i, lower=True)

            y = lg.solve_triangular(L, B_i.T, lower=True)

            new_factor = (lg.solve_triangular(L.T, y, lower=False)).T

            eps = max(eps, np.linalg.norm(new_factor - factors[i]) / np.linalg.norm(factors[i]))

            ad_prod *= new_factor.T @ new_factor

            factors[i] = new_factor

            permutat[i], permutat[-1] = permutat[-1], permutat[i]

        it += 1

    return factors, it

# Create Tensor

In [220]:
sizes = np.array((10, 11, 12, 13, 14))
T = np.zeros(sizes)
for i in range(sizes[0]):
    for j in range(sizes[1]):
        for k in range(sizes[2]):
            for m in range(sizes[3]):
                for f in range(sizes[4]):
                    T[i, j, k, m, f] = np.sin(i + j + k + m + f)

# Run fast_ALS

In [232]:
rank = 5
start_time = time.time()
factors, it = fast_ALS(T, rank)
print(time.time() - start_time)

0.13037514686584473


In [233]:
len_dim = len(sizes)
sweep_l = np.ones(rank).reshape(1, -1)

for j in range(len_dim):
    sweep_l = lg.khatri_rao(sweep_l, factors[j])

print(np.linalg.norm(np.sum(sweep_l, axis=1) - T.reshape(-1,)) / np.linalg.norm(T.reshape(-1,)))

3.219549474212735e-10


# Run ALS

In [234]:
rank = 5
start_time = time.time()
factors, it = ALS(T, rank)
print(time.time() - start_time)

0.26180052757263184


In [235]:
len_dim = len(sizes)
sweep_l = np.ones(rank).reshape(1, -1)

for j in range(len_dim):
    sweep_l = lg.khatri_rao(sweep_l, factors[j])

print(np.linalg.norm(np.sum(sweep_l, axis=1) - T.reshape(-1,)) / np.linalg.norm(T.reshape(-1,)))

3.24866474547224e-10
